Summary
# 1 - API: coordinates + weather data

# 2 - Top-5 cities Map

# 3 - Booking.com Scraping

# 4 - Top-20 hotels Map

# 5 - AWS S3 Bucket

# 6 - AWS RDS Instance

# 1 - API: coordinates + weather data

In [1]:
# Importing libraries
import requests
import pandas as pd
import json
import numpy as np

In [2]:
# Creating list of cities
list_cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", 
"Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon",
"Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [4]:
data = []

API_KEY = 'XXXX'
units = 'metric'
exclusion ='current,minutely,hourly,alerts'

for city in list_cities:
    result = requests.get('https://nominatim.openstreetmap.org/search?', params={'format':'json','country' : 'France','q' : city,'limit':1})
    city_latitude = result.json()[0]['lat']
    city_longitude = result.json()[0]['lon']

    for i in range(1,8):
        result2 = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={city_latitude}&lon={city_longitude}&exclude={exclusion}&units=metric&appid={API_KEY}")
        felt_temperature = result2.json()['daily'][i]['feels_like']['day']
        rain_probability = result2.json()['daily'][i]['pop']
        day = i
        data.append([city, city_latitude,city_longitude, felt_temperature, rain_probability, day])

df = pd.DataFrame(data=data, columns=('city', 'city_latitude','city_longitude', 'felt_temperature', 'rain_probability', 'day'))
df = df.groupby(by='city').agg({'city_latitude': 'max', 'city_longitude' : 'max', 'felt_temperature' : 'mean','rain_probability' : 'mean'}).reset_index()
df.insert(0, "city_id", df.index)
df["city_latitude"], df["city_longitude"] = df["city_latitude"].astype(float), df["city_longitude"].astype(float)
df

,city_id,city,city_latitude,city_longitude,felt_temperature,rain_probability
0,0,Aigues Mortes,43.565823,4.191284,9.524286,0.074286
1,1,Aix en Provence,43.529842,5.447474,9.855714,0.015714
2,2,Amiens,49.894171,2.295695,8.794286,0.001429
3,3,Annecy,45.899235,6.128885,7.994286,0.000000
4,4,Ariege,42.945537,1.406554,6.391429,0.160000
5,5,Avignon,43.949249,4.805901,9.071429,0.000000
6,6,Bayeux,49.276462,-0.702474,9.367143,0.207143
7,7,Bayonne,43.494514,-1.473666,10.731429,0.218571
8,8,Besancon,47.238022,6.024362,8.064286,0.000000
9,9,Biarritz,43.471144,-1.552727,9.920000,0.224286


In [5]:
# Sorting by the lowest probability of rain, and then by the higher temperature (in case mean rain probability was equal for 2 of the best 5 cities, but not the case here)
# I do not like walking under the rain, while if it is cold, you just need multiple layers of clothing!
df_meteo = df.sort_values(by=['rain_probability','felt_temperature'], ascending=[True, False])
df_meteo = df_meteo.reset_index(drop=True)
df_meteo

,city_id,city,city_latitude,city_longitude,felt_temperature,rain_probability
0,19,Grenoble,45.187560,5.735782,10.958571,0.000000
1,10,Bormes les Mimosas,43.150697,6.341928,10.371429,0.000000
2,5,Avignon,43.949249,4.805901,9.071429,0.000000
3,18,Gorges du Verdon,43.749656,6.328562,8.218571,0.000000
4,17,Eguisheim,48.044797,7.307962,8.094286,0.000000
5,8,Besancon,47.238022,6.024362,8.064286,0.000000
6,3,Annecy,45.899235,6.128885,7.994286,0.000000
7,23,Lyon,45.757814,4.832011,7.972857,0.000000
8,15,Colmar,48.077752,7.357964,7.828571,0.000000
9,32,Strasbourg,48.584614,7.750713,7.001429,0.000000


In [6]:
# Replacing accents and specific characters to make sure we get correspondance for further merge
df_meteo["city"] = [x.lower() for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("  ", " ") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("-", " ") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("ç", "c") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("œ", "oe") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("saint", "st") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("é", "e") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("è", "e") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("ê", "e") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("â", "a") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("î", "i") for x in df_meteo["city"]]
df_meteo["city"] = [x.replace("le ", "") for x in df_meteo["city"]]

In [7]:
df_meteo.to_csv('df_meteo.csv')

# 2 Top-5 cities Map

In [8]:
# Importing library to make plots
import plotly.express as px

In [9]:
# Creating a new dataframe with only the Top-5 cities for the next 7 days and displaying a map
df_meteo_map = df_meteo.head(5)

fig = px.scatter_mapbox(df_meteo_map,
                        lat = 'city_latitude',
                        lon = 'city_longitude',
                        color = 'felt_temperature',
                        color_continuous_scale = px.colors.diverging.Portland,
                        size = 'felt_temperature',
                        size_max = 30,
                        hover_name = 'city')

fig.update_layout(title='Weather map: Top-5 cities for the next 7 days',
                  title_x = 0.5,
                  width = 1000,
                  height = 800,
                  template = 'plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 2, "lat" : 47}, "zoom": 5},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80})

fig.show()

# 3 - Booking.com Scraping

In [ ]:
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

# Creating the spider
class BookingSpider(scrapy.Spider):
    name = "booking"
    start_urls = ['https://www.booking.com/']

    # Parse function for form request: getting data and looping over cities from csv file
    def parse(self, response):

        df_meteo = pd.read_csv("df_meteo.csv")
        city_list = [x for x in df_meteo["city"]]

        for city in city_list:
            yield scrapy.FormRequest.from_response(        
                    response,
                    formdata={'ss': city},
                    callback=self.after_search
                    )


    # Callback used after requesting cities, to get specific data about hotels
    def after_search(self, response):
        blocks = response.xpath('//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div')
        for block in blocks:
            dico = {
                    "city" : block.xpath('//*[@id="right"]/div[1]/div/div/div/h1/text()').get().split(":")[0]
                    ,
                    "hotel_name" : block.xpath('div[1]/div[2]/div/div/div/div[1]/div/div[1]/div/h3/a/div[1]/text()').get()
                    ,   
                    "score" : block.xpath('div[1]/div[2]/div/div/div/div[2]/div[1]/div/a/span/div/div[1]/text()').get()
                    ,
                    "description" : block.xpath('div[1]/div[2]/div/div/div/div[1]/div/div[4]/text()').get()
                    ,
                    "description2" : block.xpath('div[1]/div[2]/div/div/div/div[1]/div/div[3]/text()').get()
                    ,      
                    "url" : block.xpath('div[1]/div[2]/div/div/div/div[1]/div/div[1]/div/h3/a[contains(@href,"com")][1]/@href').get()
                    }

            # Initiating method to get GPS coordinates from hotels pages
            try: 
                yield response.follow(url = block.xpath('div[1]/div[2]/div/div/div/div[1]/div/div[1]/div/h3/a[contains(@href,"com")][1]/@href').get(),
                                    callback = self.coordinates, 
                                    cb_kwargs = {'dico': dico}
                                    ) 
            except: 
                yield dico
                print('error')
    
    # Method to add coordinates as new key on the dictionary 
    def coordinates(self, response, dico):
        try:
            coord = response.css('a.jq_tooltip.loc_block_link_underline_fix.bui-link.show_on_map_hp_link.show_map_hp_link').attrib['data-atlas-latlng']
            coord = coord.split(',')
            dico['latitude'] = float(coord[0])
            dico['longitude'] = float(coord[1])

            yield dico
        except:
            yield dico
            print('error')


# Name of the file where the results will be saved
filename = "booking-data.json"

# Deleting previous file if one with same name already existed (Scrapy would concatenate the last and new results otherwise)
if filename in os.listdir('requestfolder'):
        os.remove('requestfolder/' + filename)


# Declaring the crawling process
process = CrawlerProcess(settings = {
    # Simulating a browser on an OS
    'USER_AGENT': 'Chrome/97.0',
    # Displaying regular errors
    'LOG_LEVEL': logging.INFO,
    # Simulating human behavior
    "AUTOTHROTTLE_ENABLED": True,
    # Specifying where the file will be stored and its format
    'FEEDS': {
        'requestfolder/' + filename : {"format": "json"},
    }
})


# Starting the crawling process
process.crawl(BookingSpider)
process.start()

# 4 - Top-20 hotels Map

In [10]:
# Importing libraries for data manipulaion
import pandas as pd
import json

In [11]:
# Importing dataset and keeping only valid lines (dropping empty lines from the scrap)
df_booking = pd.read_json("booking-data.json")
df_booking = df_booking[~df_booking["hotel_name"].isna()]

In [12]:
# Creating a "long_description" and a "short_description" columns 
df_booking['long_description'] = df_booking['description'].str.cat(df_booking['description2'], na_rep="")
df_booking['short_description'] = df_booking['long_description'].str[:25]
useless_cols = ["description", "description2"]
df_booking.drop(useless_cols, axis=1, inplace=True)
df_booking

,city,hotel_name,score,url,latitude,longitude,long_description,short_description
238,Bayonne,"Vue rivière super centre Bayonne, design 60 mt",9.0,https://www.booking.com/hotel/fr/vue-riviere-s...,43.492213,-1.473737,"Vue rivière super centre Bayonne, design 60 mt...",Vue rivière super centre
239,Le Mont Saint Michel,Le Marquis De La Guintre,8.8,https://www.booking.com/hotel/fr/le-marquis-de...,48.624865,-1.445342,"Located in Courtils, Le Marquis De La Guintre ...","Located in Courtils, Le M"
240,Grenoble,Hotel Mercure Grenoble Centre Président,7.6,https://www.booking.com/hotel/fr/grand-mercure...,45.178494,5.720298,Hôtel Mercure Grenoble Président is located 10...,Hôtel Mercure Grenoble Pr
241,Cassis,LA FALAISE PARADIS Vue mer 180°,7.7,https://www.booking.com/hotel/fr/la-falaise-pa...,43.215295,5.546231,"Offering barbecue facilities and sea view, LA ...",Offering barbecue facilit
242,Bormes-les-Mimosas,SELECT'SO HOME - Résidence Le Poséidon - Mazet...,8.0,https://www.booking.com/hotel/fr/poseidon-maze...,43.124150,6.351192,SELECT'SO HOME - Résidence Le Poséidon - Mazet...,SELECT'SO HOME - Résidenc
...,...,...,...,...,...,...,...,...
1354,La Rochelle,La Mariennée,7.9,https://www.booking.com/hotel/fr/la-mariennee....,46.149583,-1.144550,"La Mariennée is set in La Rochelle, 2.1 km fro...",La Mariennée is set in La
1355,La Rochelle,La Belle Amarre,9.6,https://www.booking.com/hotel/fr/la-belle-amar...,46.158662,-1.155056,Located in La Rochelle and with Concurrence re...,Located in La Rochelle an
1356,La Rochelle,Hôtel La Monnaie Art & Spa,8.6,https://www.booking.com/hotel/fr/de-la-monnaie...,46.156301,-1.158108,"Ideally set in the historic La Rochelle, this ...",Ideally set in the histor
1357,La Rochelle,Hôtel de l'Océan,8.1,https://www.booking.com/hotel/fr/de-l-ocean-la...,46.157020,-1.154128,"Built in 1646, Hotel de L’Ocean is located on ...","Built in 1646, Hotel de L"


In [13]:
# Replacing accents and specific characters to make sure we get correspondance when merging df_meteo and df_booking
df_booking["city"] = [x.lower() for x in df_booking["city"]]
df_booking["city"] = [x.replace("  ", " ") for x in df_booking["city"]]
df_booking["city"] = [x.replace("-", " ") for x in df_booking["city"]]
df_booking["city"] = [x.replace("ç", "c") for x in df_booking["city"]]
df_booking["city"] = [x.replace("œ", "oe") for x in df_booking["city"]]
df_booking["city"] = [x.replace("saint", "st") for x in df_booking["city"]]
df_booking["city"] = [x.replace("é", "e") for x in df_booking["city"]]
df_booking["city"] = [x.replace("è", "e") for x in df_booking["city"]]
df_booking["city"] = [x.replace("ê", "e") for x in df_booking["city"]]
df_booking["city"] = [x.replace("â", "a") for x in df_booking["city"]]
df_booking["city"] = [x.replace("î", "i") for x in df_booking["city"]]
df_booking["city"] = [x.replace("î", "i") for x in df_booking["city"]]
df_booking["city"] = [x.replace("le ", "") for x in df_booking["city"]]

In [14]:
# Merging both dataframes
dataset = df_meteo.merge(df_booking, on="city", how="inner")

In [15]:
# Exporting this global dataset as csv file for further use if needed
dataset.to_csv("dataset-kayak.csv")

In [16]:
# Creating a new dataframe with only the Top-20 hotels for each of the Top-5 cities
df_booking_map = df_booking[df_booking['city'].isin(df_meteo_map["city"])]
df_booking_map = df_booking_map.groupby("city").apply(lambda x: x.nlargest(20, "score")).reset_index(drop=True)
df_booking_map

,city,hotel_name,score,url,latitude,longitude,long_description,short_description
0,avignon,Le Clos Saluces,9.9,https://www.booking.com/hotel/fr/le-clos-saluc...,43.950193,4.810586,Offering a flowered furnished garden and a pat...,Offering a flowered furni
1,avignon,Face au Palais,9.8,https://www.booking.com/hotel/fr/le-jardin-du-...,43.950951,4.806201,Face au Palais is located across the Papal Pal...,Face au Palais is located
2,avignon,Cosy and large flat in the hypercentre of Avig...,9.8,https://www.booking.com/hotel/fr/cosy-and-larg...,43.949938,4.806563,Located in Avignon and only 200 metres from Pa...,Located in Avignon and on
3,avignon,La Maison Grivolas Appartements et Maison d'hôtes,9.7,https://www.booking.com/hotel/fr/la-maison-gri...,43.951836,4.812205,"500 metres from Papal Palace, La Maison Grivol...",500 metres from Papal Pal
4,avignon,Les Jardins de Baracane,9.7,https://www.booking.com/hotel/fr/les-jardins-d...,43.944609,4.809282,A 17th century property offering an outdoor po...,A 17th century property o
...,...,...,...,...,...,...,...,...
95,grenoble,Hotel Mercure Grenoble Centre Président,7.6,https://www.booking.com/hotel/fr/grand-mercure...,45.178494,5.720298,Hôtel Mercure Grenoble Président is located 10...,Hôtel Mercure Grenoble Pr
96,grenoble,Kyriad Grenoble Centre,7.6,https://www.booking.com/hotel/fr/qualityhotelg...,45.168200,5.711995,"Located 200 metres from Stade Lesdiguières, Ky...",Located 200 metres from S
97,grenoble,Séjours & Affaires Grenoble Marie Curie,7.6,https://www.booking.com/hotel/fr/grenoble-mari...,45.194396,5.710593,This residence is a 5-minute walk from Grenobl...,This residence is a 5-min
98,grenoble,Hôtel d’Angleterre Grenoble Hyper-Centre,7.6,https://www.booking.com/hotel/fr/tiangleterre....,45.189204,5.725372,Hôtel d’Angleterre Grenoble Hyper-Centre is se...,Hôtel d’Angleterre Grenob


In [24]:
# Displaying a map with the Top-20 hotels for the Top-5 cities
fig = px.scatter_mapbox(df_booking_map,
                        title = 'Best Hotels for the Top-5 cities',
                        lat = 'latitude',
                        lon = 'longitude',
                        color = 'score',
                        size = 'score',
                        color_continuous_scale = px.colors.sequential.algae,
                        size_max = 25,
                        hover_name = 'hotel_name',
                        hover_data = {'short_description': True}
                        )

fig.update_layout(width = 1000,
                  height = 800,
                  title_x = 0.5, 
                  template='plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "center": {"lon": 4, "lat" : 47}, "zoom": 5},
                  margin = {"l": 0, "r": 0, "b": 0, "t": 80},
                  )
fig.show()

# 5 - AWS S3 Bucket

In [29]:
# Importing library to have a programmatic access to AWS
import boto3

# Specifying credentials from AWS
ACCESS_KEY = "XXXX"
SECRET_KEY = "XXXX"

# Creating S3 session
session = boto3.Session(aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=SECRET_KEY)

In [30]:
# Setting-up a resource
s3 = session.resource("s3")

# Creating a bucket
bucket = s3.create_bucket(Bucket="bucket-kayak-project-wml")

In [33]:
# Uploading csv file to the bucket
put_object = bucket.put_object(Key="dataset-kayak.csv", Body='csv')

In [34]:
# Downloading from s3 bucket
bucket.download_file('dataset-kayak.csv', 'dataset-kayak.csv')

# 6 - AWS RDS Instance

In [47]:
# Importing libraries and modules
import psycopg2
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

In [48]:
# Define database connection variables
DBHOST = "kayak-db.c6odhjfovwji.eu-west-3.rds.amazonaws.com"
DBUSER = "XXXX"
DBPASS = "XXXX"
DBNAME = "XXXX"

# Trying to add a PORT to prevent further error but useless
PORT="5432"

In [50]:
# Connecting to my postgresSQL database
engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

In [51]:
# Initializing a sessionmaker to talk to our database and opening sessions
Session = sessionmaker(bind = engine)
session = Session()

In [ ]:
# Trying to transform the dataframes to SQL. But no authorization
df_meteo.to_sql("df_meteo_sql", engine)
df_booking.to_sql("df_booking_sql", engine)

In [53]:
# Second attempt, using a fake database, using my computer's memory to mimic what a real database would do
engine2 = create_engine("sqlite:///:memory:", echo=True)

In [54]:
# Initializing a sessionmaker to talk to our local database and opening sessions
Session = sessionmaker(bind = engine2)
session = Session()

# Transforming the dataframes to SQL
df_meteo.to_sql("df_meteo_sql", engine2)
df_booking.to_sql("df_booking_sql", engine2)

2023-02-09 15:57:31,009 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("df_meteo_sql")
2023-02-09 15:57:31,010 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-09 15:57:31,011 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("df_meteo_sql")
2023-02-09 15:57:31,012 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-09 15:57:31,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-09 15:57:31,017 INFO sqlalchemy.engine.Engine 
CREATE TABLE df_meteo_sql (
	"index" BIGINT, 
	city_id BIGINT, 
	city TEXT, 
	city_latitude FLOAT, 
	city_longitude FLOAT, 
	felt_temperature FLOAT, 
	rain_probability FLOAT
)


2023-02-09 15:57:31,018 INFO sqlalchemy.engine.Engine [no key 0.00120s] ()
2023-02-09 15:57:31,022 INFO sqlalchemy.engine.Engine CREATE INDEX ix_df_meteo_sql_index ON df_meteo_sql ("index")
2023-02-09 15:57:31,022 INFO sqlalchemy.engine.Engine [no key 0.00066s] ()
2023-02-09 15:57:31,024 INFO sqlalchemy.engine.Engine COMMIT
2023-02-09 15:57:31,028 INFO sqlalchemy.engine.Eng

875

In [57]:
# Creating 2 statements, extracting all data from both tables
selection_meteo = text("SELECT * FROM df_meteo_sql")
df_meteo = pd.read_sql(selection_meteo, engine2)

selection_booking = text("SELECT * FROM df_booking_sql")
df_booking = pd.read_sql(selection_booking, engine2)

2023-02-09 15:58:54,089 INFO sqlalchemy.engine.Engine SELECT * FROM df_meteo_sql
2023-02-09 15:58:54,090 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ()
2023-02-09 15:58:54,095 INFO sqlalchemy.engine.Engine SELECT * FROM df_booking_sql
2023-02-09 15:58:54,096 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ()
